In [41]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
import kagglehub
import xgboost as xgb # 監督式模型
from sklearn.ensemble import IsolationForest # 非監督式模型

# general setting. do not change TEST_SIZE
RANDOM_SEED = 42
TEST_SIZE = 0.3

# 結果呈現
def evaluation(y_true, y_pred, model_name="Model"):
   accuracy = accuracy_score(y_true, y_pred)
   precision = precision_score(y_true, y_pred, zero_division=0)
   recall = recall_score(y_true, y_pred)
   f1 = f1_score(y_true, y_pred)

   print(f'\n{model_name} Evaluation:')
   print('===' * 15)
   print('         Accuracy:', accuracy)
   print('  Precision Score:', precision)
   print('     Recall Score:', recall)
   print('         F1 Score:', f1)
   print("\nClassification Report:")
   print(classification_report(y_true, y_pred))

In [42]:
# load dataset（from kagglehub）
path = kagglehub.dataset_download("mlg-ulb/creditcardfraud")
data = pd.read_csv(f"{path}/creditcard.csv")
data['Class'] = data['Class'].astype(int)

# prepare data
data = data.drop(['Time'], axis=1)
data['Amount'] = StandardScaler().fit_transform(data['Amount'].values.reshape(-1, 1))

# transactions of fraud and non-fraud
fraud = data[data['Class'] == 1]
nonfraud = data[data['Class'] == 0]
print(f'Fraudulent:{len(fraud)}, non-fraudulent:{len(nonfraud)}')
print(f'the positive class (frauds) percentage: {len(fraud)}/{len(fraud) + len(nonfraud)} ({len(fraud)/(len(fraud) + len(nonfraud))*100:.3f}%)')

Fraudulent:492, non-fraudulent:284315
the positive class (frauds) percentage: 492/284807 (0.173%)


In [45]:
# Extract features and labels
x = np.asarray(data.drop(columns=['Class']))
y = np.asarray(data['Class'])

# 分成訓練與測試資料
x_train, x_test, y_train, y_test = train_test_split(
   x, y, test_size=TEST_SIZE, random_state=RANDOM_SEED)

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

# 選小的正常資料集做訓練依據
n_x_train = x_train[y_train == 0]
n_x_train = n_x_train[:1000]

# 非監督式模型IsolationForest
iso_model = IsolationForest(
    n_estimators=5,
    contamination=len(fraud)/(len(fraud)+len(nonfraud)),
    max_samples=0.825,
    max_features=1,
    random_state=RANDOM_SEED
)

iso_model.fit(n_x_train)

IsolationForest(contamination=0.001727485630620034, max_features=1,
                max_samples=0.825, n_estimators=5, random_state=42)

In [46]:
# 監督式模型XGBoost
xgb_model = xgb.XGBClassifier(
    n_estimators=100,
    max_depth=11,
    learning_rate=0.51,
    subsample=0.9,
    colsample_bytree=0.9,
    eval_metric='logloss',
    random_state=RANDOM_SEED
)

# decision_function 分數（數值越小越異常）
train_scores = iso_model.decision_function(x_train).reshape(-1, 1)
test_scores = iso_model.decision_function(x_test).reshape(-1, 1)

# 把 anomaly score 當作新的特徵加到 XGBoost 特徵中
x_train_aug = np.hstack([x_train, train_scores])
x_test_aug = np.hstack([x_test, test_scores])

xgb_model.fit(x_train_aug, y_train)

# 預測與結果
y_pred = xgb_model.predict(x_test_aug)
evaluation(y_test, y_pred, model_name="Hybrid model")


Hybrid model Evaluation:
         Accuracy: 0.9996839998595555
  Precision Score: 0.9658119658119658
     Recall Score: 0.8308823529411765
         F1 Score: 0.8932806324110671

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85307
           1       0.97      0.83      0.89       136

    accuracy                           1.00     85443
   macro avg       0.98      0.92      0.95     85443
weighted avg       1.00      1.00      1.00     85443

